In [1]:
pip install dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=260eafec289121b129df48cbab82fad6f84d9a6722ae1f27c4763cd549936bad
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [5]:
import pandas as pd
from transformers import AutoModelForTokenClassification, AutoTokenizer, TrainingArguments, Trainer
from seqeval.metrics import classification_report, precision_score, recall_score, f1_score
from datasets import Dataset
import torch

train_path = 'train-00000-of-00001mt.parquet'
valid_path = 'validation-00000-of-00001mt.parquet'
test_path = 'test-00000-of-00001mt.parquet'

train_df = pd.read_parquet(train_path)
valid_df = pd.read_parquet(valid_path)
test_df = pd.read_parquet(test_path)

print("Train sample:", train_df.head())

id2label = {
    0: "O",
    1: "B-PER",
    2: "I-PER",
    3: "B-ORG",
    4: "I-ORG",
    5: "B-LOC",
    6: "I-LOC"
}
num_labels = 7

label2id = {v: k for k, v in id2label.items()}


train_dataset = Dataset.from_pandas(train_df)

valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

tokenizer = AutoTokenizer.from_pretrained("vinai/xphonebert-base", add_prefix_space=True)

def preprocess_function(examples):

    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        max_length=512,
        is_split_into_words=True
    )


    word_ids = tokenized_inputs.word_ids()
    aligned_labels = []
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            aligned_labels.append(-100)
        elif word_idx != previous_word_idx:
            aligned_labels.append(examples["ner_tags"][word_idx])
        else:
            aligned_labels.append(-100)
        previous_word_idx = word_idx

    tokenized_inputs["labels"] = aligned_labels
    return tokenized_inputs



def compute_metrics(pred):

    predictions = pred.predictions.argmax(axis=-1)
    labels = pred.label_ids


    y_pred = [[id2label[p] for p, l in zip(prediction, label) if l != -100]
              for prediction, label in zip(predictions, labels)]
    y_true = [[id2label[l] for p, l in zip(prediction, label) if l != -100]
              for prediction, label in zip(predictions, labels)]


    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


tokenized_train = train_dataset.map(preprocess_function, batched=False)
tokenized_valid = valid_dataset.map(preprocess_function, batched=False)
tokenized_test = test_dataset.map(preprocess_function, batched=False)


if 'ner_tags' in train_df.columns and not train_df.empty:
    all_labels = set(label for tags in train_df['ner_tags'] if isinstance(tags, list) for label in tags)
    num_labels = len(all_labels)
else:
    raise ValueError("`ner_tags` column is missing or empty in the train dataset.")


print("Number of unique labels:", num_labels)


model = AutoModelForTokenClassification.from_pretrained(
    "vinai/xphonebert-base",
    num_labels=7,
    id2label={
        0: "O", 1: "B-PER", 2: "I-PER",
        3: "B-ORG", 4: "I-ORG", 5: "B-LOC", 6: "I-LOC"
    },
    label2id={
        "O": 0, "B-PER": 1, "I-PER": 2,
        "B-ORG": 3, "I-ORG": 4, "B-LOC": 5, "I-LOC": 6
    }
)

model = AutoModelForTokenClassification.from_pretrained(
    "vinai/xphonebert-base",
    num_labels=7,
    id2label=id2label,
    label2id=label2id
)


training_args = TrainingArguments(
    output_dir="./xphonebert_ner_results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)



Train sample:                                               tokens  \
0               [RINDIRIZZA, Ġwann, Marija, Vianney]   
1  [Rebbieħ, tal-premju, tal-Għarbi, ,, f’Lulju, ...   
2              [Hu, ħaseb, li, kien, wasal, Asja, .]   
3                [Glen, De, Boeck, (, Nov, 2012–, )]   
4                  [RINDIRIZZA, Cercle, Brugge, KSV]   

                            ner_tags  \
0                       [0, 1, 2, 2]   
1  [0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0]   
2              [0, 0, 0, 0, 0, 5, 0]   
3              [1, 2, 2, 0, 0, 0, 0]   
4                       [0, 3, 4, 4]   

                                          langs                        spans  
0                              [mt, mt, mt, mt]  [PER: Ġwann Marija Vianney]  
1  [mt, mt, mt, mt, mt, mt, mt, mt, mt, mt, mt]                  [LOC: Ruma]  
2                  [mt, mt, mt, mt, mt, mt, mt]                  [LOC: Asja]  
3                  [mt, mt, mt, mt, mt, mt, mt]         [PER: Glen De Boeck]  
4            

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/40.8k [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Number of unique labels: 0


pytorch_model.bin:   0%|          | 0.00/350M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/xphonebert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/xphonebert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-419319c57fed>:155: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer

model.safetensors:   0%|          | 0.00/350M [00:00<?, ?B/s]

In [6]:


for idx, row in train_df.iterrows():
    if len(row["tokens"]) != len(row["ner_tags"]):
        print(f"Row {idx} mismatch: {len(row['tokens'])} tokens vs {len(row['ner_tags'])} ner_tags")
train_df = train_df[train_df.apply(lambda x: len(x['tokens']) == len(x['ner_tags']), axis=1)]

train_df = train_df[train_df.apply(lambda x: len(x["tokens"]) == len(x["ner_tags"]), axis=1)]
valid_df = valid_df[valid_df.apply(lambda x: len(x["tokens"]) == len(x["ner_tags"]), axis=1)]
test_df = test_df[test_df.apply(lambda x: len(x["tokens"]) == len(x["ner_tags"]), axis=1)]


train_dataset = Dataset.from_pandas(train_df).map(preprocess_function, batched=False)
valid_dataset = Dataset.from_pandas(valid_df).map(preprocess_function, batched=False)
test_dataset = Dataset.from_pandas(test_df).map(preprocess_function, batched=False)

for example in train_dataset:
    input_len = len(example["input_ids"])
    label_len = len(example["labels"])
    if input_len != label_len:
        print(f"Mismatch: input_ids={input_len}, labels={label_len}")


trainer.train()


results = trainer.evaluate()
print("Validation Results:", results)

predictions = trainer.predict(test_dataset)
predicted_labels = torch.argmax(torch.tensor(predictions.predictions), dim=-1)

final_predictions = [[id2label[label.item()] for label in seq] for seq in predicted_labels]

for tokens, preds in zip(test_df['tokens'].head(5), final_predictions[:5]):
    print("Tokens:", tokens)
    print("Predicted Tags:", preds)
    print("---")


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,1.435700,1.524209,0.000000,0.000000,0.000000
2,1.479700,1.344322,0.000000,0.000000,0.000000
3,1.193000,1.249408,0.000000,0.000000,0.000000


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Results: {'eval_loss': 1.24940824508667, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_runtime': 165.6196, 'eval_samples_per_second': 0.604, 'eval_steps_per_second': 0.078, 'epoch': 3.0}
Tokens: ['Emerson' 'Ferreira' 'da' 'Rosa']
Predicted Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'